In [2]:
# tao df2
import re
import pandas as pd

path = 'C:/Users/ngocb/Downloads/test_rex.xlsx'
df_trans = pd.read_excel(path)
arr_set_data = set()
num_row_data=0
num_row_process = 0
for col in df_trans.columns:
	df_trans[col] = df_trans[col].astype(str)
	lst_string_column = df_trans[col].to_list()
	num_row_data += len(lst_string_column)
	string = '\n'.join(lst_string_column)
	#thiết lập pattern
	#pattern = re.compile(r'^(?:(?:\w|\s|\b|\n){0,2})(?:(?:[0-9]{2}\/){2}[0-9]{2}[\s][0-9:]+[\s][A-Z]+[^\S][0-9]+[-0-9]{2}[\s]+(?:F|P)[\s][0-9]+[\s][0-9A-Z]+[\s]*(?:VN07)[\s]+(?:[0-9A-Z\s\/#\\]){2,8}[\s]+[0-9-.]*[\s]*(?:EA|CS|NA|N\/A|#NA|#N\/A){0,1}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9\s]{10})', re.MULTILINE)
	pattern = re.compile(r'^(?=.*?(?:\d{2}\/){2}(?:\d{2}))(?=.*?(?:(?:\d{2}:){2}\d{2})(?=[\sA-Z]))(?=.*?(?:\d+-\d{1})(?=\s+(?:F|P)))(?=.*?(?:F|P)(?=\s\d{6,12}))(?=.*?(?:\d{6,12})(?=\s{1,5}[A-Z0-9]{6,14}))(?=.*?(?:[A-Z0-9]{6,14})(?=\s{1,20}VN07))(?=.*?(?<=VN07)(?:\s+[A-Z0-9#\/\\\s]{2,14})).{30,}(?:\s|\n|\b){0,50}(?:Tech=){1}[A-Z0-9]{2,15}', re.MULTILINE)
	pattern_check_right = re.compile(r'(?=.*?(?<=VN07)(;[A-Z0-9#\/\\\s]+))(?=.*?([0-9.,]+)(?=;(?:CS|EA)))(?=.*?(?<=(?:CS|EA))(;[A-Z0-9]{2,15}))')
	patern_check_left = re.compile(r'(?=.*?(?<=[A-Z])(;\d+-\d{1}))(?=.*?(?<=(?:\d{2}\/){2}\d{2})(;(?:\d{2}:){2}\d{2}))(?=.*?(?<=(?:F|P))(;\d{2,20}))(?=.*?(?:\d+;)(?=[A-Z0-9]*))')

	list_trans = pattern.findall(string)
	list_trans_remove_duplicate = set(list_trans)
	num_row_process += len(list_trans)
	for i, line in enumerate(list_trans_remove_duplicate):
		#bỏ ký tự \n, Tech=
		row = re.sub(r'(\n|Tech=)', '', line).strip()
		#tìm index VN07
		index_vn07 = re.search(r'VN07', row).span()[0]
		#cắt chuỗi thành 2 phần trái, phải. để xử lý space location
		left_row = row[:index_vn07]
		right_row = row[index_vn07:]
		# with left
		left = re.sub(r'\s{1,}', ';', left_row)
		#with right
		# right1 = re.sub(r'(?<=[A-Z0-9])(\s{1})(?=[0-9A-Z])', '', right_row)
		right2 = re.sub(r'\s{2,}', ';', right_row)
		right = re.sub(r'\s{1,}', '', right2)
		#thêm đơn vị cho RPM
		# regex tìm chữ P theo sau nó phải là dấu ; và trước nó cũng là dấu ;
		# regex tìm chữ F theo sau nó phải là dấu ; và trước nó cũng là dấu ;. Trường hợp CASXFR không có đơn vị
		pattern_p = re.compile(r'(?:(?<=;)(P)(?=;))')
		pattern_f = re.compile(r'(?:(?<=;)(F)(?=;))')
		match_p = pattern_p.search(left)
		match_f = pattern_f.search(left)
		arr_right = right.split(';')
		if (len(arr_right) == 4) and (match_p is not None):
			arr_right.insert(-1, 'EA')
			right = ';'.join(arr_right)
		elif (len(arr_right) == 4) and (match_f is not None):
			arr_right.insert(-1, 'CS')
			right = ';'.join(arr_right)
		#ket qua
		string_final = left + right
		#check kết quả thêm data vào arr kết quả
		match_right = pattern_check_right.match(right)
		match_left = patern_check_left.match(left)
		if match_left and match_right:
			arr_set_data.add(string_final)
	   
print('Tổng số dòng data ban đầu: ', num_row_data)
print('Tổng số dòng cần xử lý: ', num_row_process)
print('Tổng số dòng cần lấy: ', len(arr_set_data))
print('Tổng số dòng data trùng nhau: ', num_row_process - len(arr_set_data))
arr_set_data

Tổng số dòng data ban đầu:  5240075
Tổng số dòng cần xử lý:  807865
Tổng số dòng cần lấy:  801949
Tổng số dòng data trùng nhau:  5916


{'11/20/24;12:35:00;DEPSUL;0010009772009019614-0;F;80780085;43250395D6;VN07;FK28A;50;CS;BID411',
 '11/08/24;09:47:06;DEPSUL;0010009772008997294-6;F;80815421;43130395P4;VN07;STAM;32;CS;BID258',
 '11/16/24;23:00:05;DEPSUL;0010009772009019158-9;F;80779871;43210395D6;VN07;ST12;50;CS;BID231',
 '11/11/24;04:39:44;PICKUL;0010009772008998623-3;F;80796707;43150395Z3;VN07;CONT047;48;CS;BID511',
 '11/12/24;07:47:30;PICKUL;0010009772008998868-8;F;80796705;43170395Z0;VN07;FF12A;48;CS;BID032',
 '11/11/24;07:50:49;PICKUL;0010009772009037721-1;F;80779084;43140395C6;VN07;FF20A;50;CS;BID366',
 '11/21/24;10:12:33;PICKUL;0010009772009014305-2;F;80811898;43250395T2;VN07;CONT045;44;CS;BID379',
 '11/18/24;13:40:16;LINUSE;0030009772002771089-4;P;90641517;0395412550;VN07;ULINDC;792;EA;BID905',
 '11/14/24;07:54:47;PICKUL;0030009772002772449-5;P;21138551;0395412790;VN07;G614E;345;EA;BID527',
 '11/12/24;04:24:28;DEPSUL;0010009772009036613-0;F;80831264;43160395K9;VN07;C030;34;CS;BID511',
 '11/17/24;03:08:44;DEPSUL

In [3]:
import pandas as pd
lst_total_2 = []
for line in arr_set_data:
    lst_line = line.split(';')
    lst_total_2.append(lst_line)

# transposed = list(zip(*lst_total))

columns = ['date', 'time', 'type', 'ulid', 'class', 'gcas', 'batch', 'plan', 'location', 'qty', 'unit', 'T#']
df_178687 = pd.DataFrame(lst_total_2, columns=columns)
# df_178687.to_excel('df.xlsx')



In [6]:
df_178687.sample(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
664715,11/13/24,18:31:44,PICKUL,0010009772008980864-1,P,21151929,0395401398,VN07,DA05B2,4110,EA,BID909
498517,11/07/24,13:06:56,DEPSUL,0010009772008967839-8,F,80796705,43110395Z4,VN07,CONT053,48,CS,BID252
2554,11/02/24,23:31:41,PICKUL,0010009772008954975-9,F,80773336,43070395C9,VN07,ST11,40,CS,BID410
177110,11/12/24,06:25:47,DEPSUL,0010009772008996858-1,F,80779858,43170395O0,VN07,CONT027,45,CS,BID276
466251,11/28/24,23:12:05,PICKUL,0010009772009629813-8,F,80814142,43330395Z0,VN07,FF28A,48,CS,BID347
512174,11/27/24,18:38:26,PICKUL,0030009772002781554-4,P,21164989,0395414602,VN07,ST02,270,EA,BID031
595943,11/02/24,11:58:03,LINUSE,0010009772008534226-2,P,21019627,0395409086,VN07,ULINAU,80,EA,BID114
19083,11/19/24,07:35:21,PICKUL,0010009772009004349-9,F,80792030,43230395G2,VN07,ST09,56,CS,BID999
289737,11/16/24,04:08:13,DEPSUL,0010009772009033092-6,F,80779083,43200395C6,VN07,WH2FB,50,CS,BID372
133978,11/21/24,13:45:35,DEPSUL,0030009772002778624-0,P,21192826,0395413999,VN07,ST04,1500,EA,BID363


In [4]:
df_sort = df_178687.sort_values(['date', 'time'], ascending=True)
df_sort.tail(10)

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#
178868,12/01/24,05:57:45,LINTAK,0010009772008990914-0,F,80809262,43350395N2,VN07,AK,40,CS,BID064
337872,12/01/24,05:57:48,LINTAK,0010009772008990913-3,F,80809262,43350395N2,VN07,AK,40,CS,BID064
260245,12/01/24,05:57:50,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
533231,12/01/24,05:57:50,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID064
516926,12/01/24,05:58:05,PICKUL,0010009772008990913-3,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
706203,12/01/24,05:58:05,PICKUL,0010009772008990914-0,F,80809262,43350395N2,VN07,STAK,40,CS,BID384
419492,12/01/24,05:58:16,LINTAK,0010009772008991987-3,F,80813651,43360395L0,VN07,AE,50,CS,BID182
247201,12/01/24,05:58:26,DEPSUL,0010009772008991987-3,F,80813651,43360395L0,VN07,STAE,50,CS,BID182
248206,12/01/24,05:59:52,DEPSUL,0010009772008990914-0,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384
321088,12/01/24,05:59:52,DEPSUL,0010009772008990913-3,F,80809262,43350395N2,VN07,FI25A,40,CS,BID384


In [9]:
check_columns = df_178687['class'].unique()
check_columns.sort()
check_columns

array(['F', 'P'], dtype=object)

In [10]:
df_178687[df_178687['ulid'] == '0010009772008759523-9']
#0010009772008683057-7

,date,time,type,ulid,class,gcas,batch,plan,location,qty,unit,T#


### So sánh 2 dataframe

In [53]:
# df_dff = pd.merge(left=df_180430,
#                   right=df_178687,
#                   on=['Time', 'Ulid'],
#                   how='left',
#                   indicator=True)


In [54]:
# df_khac = df_dff[df_dff['_merge'].isin(['left_only'])]
# # df_khac.to_excel('df_dff.xlsx')
# df_khac